In [1]:
%load_ext sql
%config SqlMagic.feedback = False

In [2]:
from os import environ
if environ.get("UDACITY_DATA_ENGINEER_POSTGRES_INSTANCE"):
    host = environ.get("UDACITY_DATA_ENGINEER_POSTGRES_INSTANCE")
    user = environ.get("UDACITY_DATA_ENGINEER_POSTGRES_USER")
    password = environ.get("UDACITY_DATA_ENGINEER_POSTGRES_PASSWORD")

conn_string = "postgresql://{}:{}@{}/sparkifydb".format(user, password, host)

In [ ]:
%sql $conn_string SELECT * FROM songplays LIMIT 5;

In [ ]:
%sql $conn_string SELECT * FROM users LIMIT 5;

In [ ]:
%sql $conn_string SELECT * FROM songs LIMIT 5;

In [ ]:
%sql $conn_string SELECT * FROM artists LIMIT 5;

In [4]:
%sql $conn_string SELECT * FROM time LIMIT 5;

start_time,hour,day,week,month,year,weekday
2018-11-22 00:03:52.796000,0,22,47,11,2018,3
2018-11-22 00:09:10.796000,0,22,47,11,2018,3
2018-11-22 00:12:06.796000,0,22,47,11,2018,3
2018-11-22 00:15:41.796000,0,22,47,11,2018,3
2018-11-22 00:23:40.796000,0,22,47,11,2018,3


## REMEMBER: Restart this notebook to close connection to `sparkifydb`
Each time you run the cells above, remember to restart this notebook to close the connection to your database. Otherwise, you won't be able to run your code in `create_tables.py`, `etl.py`, or `etl.ipynb` files since you can't make multiple connections to the same database (in this case, sparkifydb).